In [3]:
import requests
import json
import re
from tqdm import tqdm

# API URL for Fireworks model completion endpoint
api_url = "https://api.fireworks.ai/inference/v1/completions"

# Authorization token (replace with your actual Fireworks API Key)
api_key = "fw_3ZcbBTZRScdUSrVf2NuwPRJ2"

# The deployed model identifier (from your output)
model_id = "accounts/myonlineretail2023-b215a8/deployedModels/d901106fd727401eb7855c700ea69b29-8a5acd69"

# Function to query the model
def query_model(prompt):
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }

    data = {
        "model": model_id,
        "prompt": prompt,
        "max_tokens": 100
    }

    response = requests.post(api_url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()["choices"][0]["text"]
    else:
        print(f"Error querying model: {response.status_code}, {response.text}")
        return None


def extract_number_from_output(output):
    match = re.search(r"####\s*([\d,]+(\.\d+)?)", output)
    if match:
        return float(match.group(1).replace(',', ''))
    return None

# Load the validation dataset (GSM8K) from a JSONL file
def load_validation_dataset(file_path):
    dataset = []
    with open(file_path, 'r') as f:
        for line in f:
            dataset.append(json.loads(line))
    return dataset



correct = 0
total = 0
reasoning_outputs = []

# Load the validation dataset and process it
validation_file_path = "gsm8k_val.jsonl"
reasoning_output_file = "reasoning_outputs.jsonl"

# File to save the progress
progress_file = "correct_progress_mistral.txt"

dataset = load_validation_dataset(validation_file_path)

for i, example in tqdm(enumerate(dataset)):
    example = dataset[i]
    prompt = "Give the final answer as #### <answer> where answer is the final numerical value ," + example['question']  # Use 'prompt' field to query the model
    try:
        ground_truth = float(re.search(r"####\s*(\d+(\.\d+)?)", example['answer']).group(1))
    except:
        continue

    if ground_truth is None:
        continue
    
    # Query the model with the prompt
    model_output = query_model(prompt)
    
    if model_output is None:
        continue

    # Extract the predicted final answer from the output
    predicted_number = extract_number_from_output(model_output)
    
    if predicted_number is None:
        continue
    else:
        reasoning_outputs.append({
            "question": example["question"],
            "model_output": model_output,
            "ground_truth": ground_truth
        })
        
        # Compare the predicted number with the ground truth
        if predicted_number == ground_truth:
            correct += 1
            print(f"Match: Predicted {predicted_number}, Expected {ground_truth}")
            print(f"Current correct count: {correct}")  # Print the current value of correct after each match
        else:
            print(f"Mismatch: Predicted {predicted_number}, Expected {ground_truth}")

        total += 1

    # Save progress to a file
    with open(progress_file, "w") as f:
        f.write(f"Correct: {correct}, Total: {total}\n")

# Save the reasoning outputs to a separate JSONL file
with open(progress_file, 'w') as f:
    for reasoning in reasoning_outputs:
        json.dump(reasoning, f)
        f.write("\n")

# Print final correct count before calculating accuracy
print(f"Final correct count: {correct}")  # Final correct count
accuracy = (correct / total) * 100
print(f"Accuracy on validation dataset: {accuracy}%")

1it [00:03,  3.47s/it]

Match: Predicted 18.0, Expected 18.0
Current correct count: 1


2it [00:04,  1.92s/it]

Match: Predicted 3.0, Expected 3.0
Current correct count: 2


5it [00:09,  1.60s/it]

...

Mismatch: Predicted 72.0, Expected 64.0


1314it [36:09,  1.80s/it]

Mismatch: Predicted 2.5, Expected 2.0


1319it [36:18,  1.65s/it]

Mismatch: Predicted 1.75, Expected 14.0
Final correct count: 230
Accuracy on validation dataset: 48.421052631578945%


In [4]:
print(f"accuracy: {accuracy}")

accuracy: 48.421052631578945
